In [71]:
import numpy as np
import pandas as pd
from scipy.special import expit
from scipy.stats import bernoulli
from sklearn.metrics import accuracy_score

In [72]:
%run LogisticRegression_with_IRLS.py

### Dataset 1

In [73]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [74]:
# Generate a synthetic dataset
np.random.seed(123)
X = np.random.normal(size=(100, 2))
y = np.random.binomial(1, sigmoid(np.dot(np.column_stack((np.ones(100), X)), np.array([1, 2, 3]))))

In [75]:
clf = LogisticRegression_with_IRLS()
clf.fit(X,y,[[0,1]])
proba = clf.predict_proba(X)
y_pred = clf.predict(X)
accuracy_score(y,y_pred)

0.68

In [76]:
proba

array([0.36972285, 0.48578891, 0.42195174, 0.35466602, 0.53847856,
       0.4470569 , 0.56627923, 0.46908775, 0.8572493 , 0.60416166,
       0.62863274, 0.38209671, 0.43209554, 0.49331574, 0.4857589 ,
       0.48645824, 0.41207518, 0.56535507, 0.5074774 , 0.41916147,
       0.4870753 , 0.46270362, 0.52729108, 0.73094716, 0.70229418,
       0.45207354, 0.56296992, 0.51513623, 0.6641347 , 0.70229767,
       0.41476608, 0.49069063, 0.67461983, 0.50075459, 0.39371935,
       0.50981319, 0.51863396, 0.24987492, 0.46838552, 0.48335809,
       0.55117551, 0.70030915, 0.48041955, 0.47280377, 0.68097985,
       0.50661179, 0.53778396, 0.46583817, 0.47252936, 0.51971292,
       0.46734373, 0.67195382, 0.49830111, 0.47833654, 0.48333503,
       0.45110832, 0.42695933, 0.5986288 , 0.40882742, 0.43747513,
       0.47950584, 0.52220791, 0.60485029, 0.59113159, 0.51925931,
       0.46301569, 0.42967259, 0.57737999, 0.42409999, 0.59193139,
       0.42914798, 0.62562631, 0.53716663, 0.58820085, 0.48018

In [77]:
y_pred

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0])

In [78]:
clf.get_params()

array([0.32658817, 0.04153298, 0.20344019])

### Dataset 2

In [79]:
n = 50
b = np.array([2.3, 4.5, 0.345])
X = np.zeros((n,3))
for i in range(n):
    X[i][0]=np.random.normal(0,1,1)
    X[i][1]=np.random.normal(3,2,1)
    X[i][2]=np.random.normal(-5,3,1)
p=(1/(1+np.exp(-(np.dot(X, b.T)))))
y = bernoulli.rvs(p, size=n)

In [80]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1])

In [81]:
clf = LogisticRegression_with_IRLS()
clf.fit(X,y)
proba = clf.predict_proba(X)
y_pred = clf.predict(X)
accuracy_score(y,y_pred)


0.88